# 정리를 하다가 말다가 해서 자세한건 책에 다 나와 았습니다.

## 6.4 일반선형모델의 평가
- 정규선형모델고 마찮가지로 모델을 평가할 때 잔차의 체크는 필수이다.
- 모집단분포가 정규분포 이외의 분포가 되면 잔차의 취급이 크게 바뀐다.
- 잔차는 데이터와 모델의 괴리를 표현하는 중요한 지표이다.

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import statsmodels.formula.api as smf
import statsmodels.api as sm

EX10)과 같은 데이터 사용

In [2]:
# 데이터를 불러와서 모델 만드는 것까지.
test_result = pd.read_csv('../input/lm-model/6-3-1-logistic-regression.csv')
mod_glm = smf.glm('result~hours', data=test_result, family=sm.families.Binomial()).fit()

### 피어슨 잔차
- 이항분포에서 **피어슨 잔차**의 계산
- $y$는 족속변수(이항확률변수, 이경우 합격여부), $N$은 시행횟수,\
  $\hat{p}$은 추측한 성공확률(mod_glm.predict())로 계산한 예측치)
#### $Pearson\,residuals = \frac{y-N\hat{p}}{\sqrt{N\hat{p}(1-\hat{p})}}$
- 여기서 하나하나의 예측 결과의 시행횟수는 당연히 1이 되기 때문에 피어슨 잔차는 다음과 같이 계산한다.
#### $Pearson\,residuals = \frac{y-\hat{p}}{\sqrt{\hat{p}(1-\hat{p})}}$

### 6.4.3 피어슨 잔차의 해석
- 피어슨 잔차의 분모에 있는 $N\hat{p}(1-\hat{p})$은 이항분포의 분산의 값과 일치한다.
  그 값에 루트를 취한 것이므로 이항분포의 표준편차로 볼 수 있다.
- 정규선형모델에서는 종속변수와 predict() 함수로 구한 예측값의 차이를 잔차로 사용했다.
  $y-\hat{p}$를 잔차로 사용하는 느낌이다.
- 피어슨 잔차는 일반 잔차를 분포의 표준편차로 나눈것으로 해석할 수 있다.
- $N$을 고정했을 때 이항분포의 분산 $Np(1-P)$가 최고로 클 때는 $p=0.5$일 때이다.
  합, 불이 반반일때 데이터가 크게 흩어질 것이라는 것.
  이때 데이터의 흩어짐이 크다해도 상정 범위 내의 '작은 흩어짐'이라고 할 수 있다.
- 반대로 $p=0.9$와 같이 합격이 거의 확실히 예측되는 경우 분산도 작아진다.
- 이때의 예측을 제외하면 큰 차이라고 간주할 수 있다.
- ***이것이 피어슨 잔차이다.***\

### 피어슨 잔차

In [3]:
# 예측한 성공확률
pred = mod_glm.predict()

# 종속변수(시험 합격여부)
y = test_result.result

# 피어슨 잔차
pearson_resid = (y-pred) / np.sqrt(pred*(1-pred))
pearson_resid.head()

0   -0.102351
1   -0.102351
2   -0.102351
3   -0.102351
4   -0.102351
Name: result, dtype: float64

In [4]:
# 모델에서 피어슨 잔차 가져오기
mod_glm.resid_pearson.head()

0   -0.102351
1   -0.102351
2   -0.102351
3   -0.102351
4   -0.102351
dtype: float64

In [5]:
# 피어슨잔차제곱합은 피어슨 카이제곱통계량이 된다.
np.sum(mod_glm.resid_pearson**2)

84.91138782569973

In [6]:
# 위의 결과는 summary() 함수에서도 출력되고, 직접 가져올 수도 있다.
mod_glm.pearson_chi2

84.91138782569973

### 6.4.5 deviance
- **deviance**는 모델의 적합도를 평가하는 지표이다.
- deviance가 크면 모델이 맞지 않다고 평가할 수 있다.
- 로지스틱 회귀의 로그우도함수를 $\log\mathcal{L}(\beta_{0},\beta_{1};N,m)$ 이라고 한다.\
  계수 $\beta_{0},\beta_{1}$ 을 바꿈으로써 우도가 변한다.\
  여기서 최대우도법을 추정한 로지스틱 회귀의 계수에 근거한 로그우도를 $\log\mathcal{L}(\beta_{glm};y)$ 라고 한다.
- 모든 합격 여부를 완전히 예측할 수 있을 때의 로그우도를 $\log\mathcal{L}(\beta_{max};y)$ 라고 한다.

### 6.4.8 deviance 잔차
- 이항분포에서 **deviance 잔차** 는 deviance 잔차제곱합이 deviance가 된다는 사실로 계산한다.

In [7]:
# 예측한 성공확률
pred = mod_glm.predict()

# 종속변수(테스트 합격여부)
y = test_result.result

# 합격여부를 완전히 예측할 수 있을 때의 로그우도와 잔차
resid_tmp = 0 - np.log(sp.stats.binom.pmf(k=y, n=1, p=pred))

# deviance 잔차
deviance_resid = np.sqrt(2*resid_tmp) * np.sign(y-pred)
deviance_resid.head()

0   -0.144369
1   -0.144369
2   -0.144369
3   -0.144369
4   -0.144369
Name: result, dtype: float64

In [8]:
# 모델에서 직접얻기
mod_glm.resid_deviance

0    -0.144369
1    -0.144369
2    -0.144369
3    -0.144369
4    -0.144369
        ...   
95    0.210216
96    0.210216
97    0.210216
98    0.210216
99    0.210216
Length: 100, dtype: float64

In [9]:
# deviance 잔차제곱합은 deviance가 된다. summary에서도 출력됨
np.sum(mod_glm.resid_deviance**2)

68.02788118117269

In [10]:
# 한방에 보임. 근데 이게 뭐 어떻다는 건지... ㅠㅠ
mod_glm.deviance

68.02788118117269

**교차 엔트로피 오차 최소화**는 **deviance 최소화**와 같고, 로지스틱 회귀의 **로그우도를 최대**로 하는 것과 같다.